In [1]:
import findspark, datetime, pytz
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import lit,substring,concat,col,aes_encrypt,aes_decrypt,base64,unbase64,count

In [2]:
agora = datetime.datetime.now(pytz.timezone('America/Sao_paulo'))
chave_crypt = 'minhachavede128b'
strBronzePath = f'../BRONZE/loja_simulada-vendas'
strSilverPath = f'../SILVER/loja_simulada-vendas'

In [3]:
# Create SparkSession
spark = (SparkSession.builder
           .appName('IngestaoSilver')
           .config("packages", "org.apache.spark:mysql-connector-java-8.0.13.jar")
           .getOrCreate()
        )

24/06/14 23:28:24 WARN Utils: Your hostname, cj resolves to a loopback address: 127.0.1.1; using 192.168.15.34 instead (on interface enp2s0)
24/06/14 23:28:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/14 23:28:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/14 23:28:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/14 23:28:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
def pegarSilverAtual(tabela):
    try:
        tempDF = (spark.read.parquet(f'{strSilverPath}-{tabela}')).cache()
        print(f'Base SILVER "{tabela}" encontrada')
        return tempDF
    except Exception as e:
        print(f'Base SILVER "{tabela}" não encontrada\n', e)
        return None

In [5]:
def pegarBronzeAtual(tabela):
    try:
        tempDF = (spark.read.parquet(f'{strBronzePath}-{tabela}')).cache()
        print(f'Base BRONZE "{tabela}" encontrada')
        return tempDF
    except Exception as e:
        print(f'Base BRONZE "{tabela}" não encontrada\n', e)
        return None

In [6]:
def enviarAlerta(tipoAlerta, mensagem):
    print(F'Encaminhando alerta de {tipoAlerta}')
    print(f'{tipoAlerta} - type: {type(mensagem)} - Mensagem:', mensagem)
    return

In [7]:
def salvarDadosDoDF(df,pathFile):
    return df.write.mode("overwrite").parquet(pathFile)

In [8]:
# TABELA CLIENTES
def clientes():
    tabela = 'clientes'
    bronzeDF = pegarBronzeAtual(tabela)
    if bronzeDF == None:
        enviarAlerta('Atencao', f'Ingestão BRONZE - SILVER base "{tabela}" interrompida, base BRONZE não encontrada')
        return
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {bronzeDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    silverDF = (bronzeDF
        .select('surrogate_key', 'id_cliente'
            # mantendo apenas os dígitos menos significativos do CPF
            , concat(lit('***'), substring('cpf', 4, 6), lit('**-**')).alias('cpf')
            # criptografia no nome do cliente
            , base64(aes_encrypt('cliente', lit(chave_crypt), lit('ECB'), lit('PKCS'))).alias('cliente')
            # , 'telefone', 'email'     # dados omitidos para a camada de destino em questão
            , 'estado', 'origem_racial', 'sexo', 'status')
        .withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    )
    silverAtual = pegarSilverAtual(tabela)
    if silverAtual == None:
        salvarDadosDoDF(silverDF, f'{strSilverPath}-{tabela}')
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {silverDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        silverBothDF = silverAtual.unionByName(silverDF).distinct()
        salvarDadosDoDF(silverBothDF, f'{strSilverPath}-{tabela}')
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {silverBothDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # # TESTE DESCRIPTOGRAFIA - INICIO
    #     bronzeDF.select('cliente').show(2)
    #     silverDF.select('cliente').show(2)
    #     clientes_silver_df_decript = (silverDF
    #         .select(aes_decrypt(unbase64('cliente'),lit(chave_crypt),lit('ECB'),lit('PKCS')).cast('string').alias('descriptado'))
    #     )
    #     bronzeDF.select('cliente').show(2)
    # # TESTE DESCRIPTOGRAFIA - FIM
    return

In [9]:
# TABELA VENDEDORES
def vendedores():
    tabela = 'vendedores'
    bronzeDF = pegarBronzeAtual(tabela)
    if bronzeDF == None:
        enviarAlerta('Atencao', f'Ingestão BRONZE - SILVER base "{tabela}" interrompida, base BRONZE não encontrada')
        return
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {bronzeDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    silverDF = (bronzeDF #.select('*').where(col('id_vendedor') < 10)  # TODO inserida para teste, retirar
        .select('surrogate_key', 'id_vendedor'
            # mantendo apenas os dígitos menos significativos do CPF
            , concat(lit('***'), substring('cpf', 4, 6), lit('**-**')).alias('cpf')
            # criptografia no nome do vendedor
            , base64(aes_encrypt('nome', lit(chave_crypt), lit('ECB'), lit('PKCS'))).alias('nome'))
            # ,'telefone','email'     # dados omitidos para a camada de destino em questão            
        .withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    )
    silverAtual = pegarSilverAtual(tabela)
    if silverAtual == None:
        salvarDadosDoDF(silverDF, f'{strSilverPath}-{tabela}')
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {silverDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        silverBothDF = silverAtual.unionByName(silverDF).distinct()
        salvarDadosDoDF(silverBothDF, f'{strSilverPath}-{tabela}')
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {silverBothDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [10]:
# TABELA PRODUTOS
def produtos():
    tabela = 'produtos'
    bronzeDF = pegarBronzeAtual(tabela)
    if bronzeDF == None:
        enviarAlerta('Atencao', f'Ingestão BRONZE - SILVER base "{tabela}" interrompida, base BRONZE não encontrada')
        return
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {bronzeDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    silverDF = (bronzeDF #.select('*').where(col('id_produto') < 10)  # TODO inserida para teste, retirar
        .select('surrogate_key', 'id_produto','produto','preco','data_carga')
        .withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    )
    silverAtual = pegarSilverAtual(tabela)
    if silverAtual == None:
        salvarDadosDoDF(silverDF, f'{strSilverPath}-{tabela}')
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {silverDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        silverBothDF = silverAtual.unionByName(silverDF).distinct()
        salvarDadosDoDF(silverBothDF, f'{strSilverPath}-{tabela}')
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {silverBothDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [11]:
# TABELA VENDAS
def vendas():
    tabela = 'vendas'
    bronzeDF = pegarBronzeAtual(tabela)
    if bronzeDF == None:
        enviarAlerta('Atencao', f'Ingestão BRONZE - SILVER base "{tabela}" interrompida, base BRONZE não encontrada')
        return
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {bronzeDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    silverDF = (bronzeDF #.select('*').where(col('id_venda') < 10)  # TODO inserida para teste, retirar
        .select('surrogate_key', 'id_venda','id_vendedor','id_cliente', 'total',col('data').alias('data_venda'))
        .withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    )
    silverAtual = pegarSilverAtual(tabela)
    if silverAtual == None:
        salvarDadosDoDF(silverDF, f'{strSilverPath}-{tabela}')
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {silverDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        silverBothDF = silverAtual.unionByName(silverDF).distinct()
        salvarDadosDoDF(silverBothDF, f'{strSilverPath}-{tabela}')
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {silverBothDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [12]:
# TABELA ITENS VENDAS
def itensVendas():
    tabela = 'itens_venda'
    bronzeDF = pegarBronzeAtual(tabela)
    if bronzeDF == None:
        enviarAlerta('Atencao', f'Ingestão BRONZE - SILVER base "{tabela}" interrompida, base BRONZE não encontrada')
        return
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {bronzeDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    silverDF = (bronzeDF #.select('*').where(col('id_venda') < 10)  # TODO inserida para teste, retirar
        .select('surrogate_key', 'id_produto','id_venda','quantidade','valor_unitario','valor_total','desconto')
        .withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    )
    silverAtual = pegarSilverAtual(tabela)
    if silverAtual == None:
        salvarDadosDoDF(silverDF, f'{strSilverPath}-{tabela}')
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {silverDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        silverBothDF = silverAtual.unionByName(silverDF).distinct()
        salvarDadosDoDF(silverBothDF, f'{strSilverPath}-{tabela}')
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {silverBothDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [13]:
def executar():
    clientes()
    vendedores()
    produtos()
    vendas()
    itensVendas()
    return

In [14]:
def main():
    try:
        enviarAlerta('Informacao', 'ingestão para base SILVER iniciada...')
        executar()
        enviarAlerta('Informacao', 'Ingestão para base SILVER concluída!!!')
    except Exception as e:
        enviarAlerta('Erro', e)
    finally:
        spark.stop()
        print('conexão spark finalizada!')

In [15]:
main()

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: ingestão para base SILVER iniciada...
Base BRONZE "clientes" encontrada
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze clientes: 250
Base SILVER "clientes" encontrada
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 780
Base BRONZE "vendedores" encontrada
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze vendedores: 10
Base SILVER "vendedores" encontrada
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 46
Base BRONZE "produtos" encontrada
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze produtos: 10
Base SILVER "produtos" encontrada
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 24
Base BRONZE "vendas" encontrada


***